In [1]:
from sklearn.neighbors import BallTree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import pickle as pk
import numpy as np


# Data preparation

In [2]:
coords = pk.load(open('../datasets/abakan_full_routes_deeptte_urban.pkl', 'rb')).edges_geo

In [3]:
len(coords)

86366

Tips:
- List your latitude coordinates before longitude coordinates.
- Check that the first number in your latitude coordinate is between -90 and 90.
- Check that the first number in your longitude coordinate is between -180 and 180.
  
To fix it we execute the following code:

In [4]:
for path in coords:
    for coord in path:
        x = coord[0]
        coord[0] = coord[1]
        coord[1] = x

In [5]:
coords

0         [[53.7087046931, 91.44752843785], [53.70874318...
2         [[53.72369937995, 91.4237220048], [53.72606540...
4         [[53.70246281705, 91.40053801445], [53.7026021...
5         [[53.738121163049996, 91.4201630593], [53.7391...
8         [[53.72837416995, 91.42058688445], [53.7288224...
                                ...                        
122355    [[53.734115891849996, 91.4402370372], [53.7347...
122359    [[53.69422431275, 91.43217182785], [53.6949184...
122360    [[53.725109522349996, 91.44607190945001], [53....
122362    [[53.707058982199996, 91.49108379335], [53.707...
122363    [[53.72535209655, 91.4557347276], [53.72629702...
Name: edges_geo, Length: 86366, dtype: object

In [6]:
raw_coords = []
for pair in coords:
    for coord in pair:
        raw_coords.append(coord)

In [7]:
for i in raw_coords:
    if (i[0] == 0):
        print(str(i))

In [31]:
raw_coords[0]

array([53.70870469, 91.44752844])

# BallTree

The Haversine equation is used to determine the distance between two points (x and y) on the Earth based on a mean spherical earth radius.  The Haversine - Distance equation is important in navigation, giving great-circle distances between two points on a sphere from their longitudes and latitudes.

<img src="https://www.vcalc.com/attachments/e6d11679-da27-11e2-8e97-bc764e04d25f/MaimiToLondon.JPG" width="400" />

See more: https://www.vcalc.com/wiki/vCalc/Haversine+-+Distance 

n_neighbors = 7853220 / 2810 = 2 794,7, it might be ok to take 2 700.

In [10]:
nbrs = NearestNeighbors(n_neighbors=2700, algorithm='ball_tree', metric='haversine').fit(raw_coords)

In [ ]:
nbrs = KNeighborsClassifier(n_neighbors=2700, algorithm='ball_tree', metric='haversine').fit(raw_coords)

In [60]:
nbrs.p

2

In [11]:
tree = BallTree(raw_coords, metric='haversine')

In [16]:
tree.node_data

sklearn.neighbors._ball_tree._memoryviewslice

# Clustering

## K-means

In [19]:
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

In [20]:
mbKMeans = MiniBatchKMeans(n_clusters = 8440, random_state = 0, batch_size=200000, max_iter=100)
mbKMeans.fit(raw_coords)
mbKMeans.cluster_centers_
df = pd.DataFrame(mbKMeans.cluster_centers_)
df.to_csv("../res/cluster_centers.csv")

In [21]:
df

,0,1
0,53.704767,91.433073
1,53.735717,91.469943
2,53.653767,91.576902
3,53.804274,91.337155
4,53.769334,91.396471
...,...,...
8435,53.702373,91.404976
8436,53.762392,91.394026
8437,53.725927,91.401658
8438,53.824557,91.343661


# Birch

In [22]:
from sklearn.cluster import Birch

In [23]:
birch = Birch(n_clusters=8440)
birch.fit(raw_coords)
df = pd.DataFrame(birch.clu, columns=['lat', 'lon'])
df.to_csv("../res/birch_subcluster_centers.csv")
df

c:\Temp\TTE\env\lib\site-packages\sklearn\cluster\_birch.py:717: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (8440). Decrease the threshold.
  warnings.warn(


,lat,lon
0,53.723426,91.42806


In [31]:
birch.

c:\Temp\TTE\env\lib\site-packages\sklearn\cluster\_birch.py:717: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (8440). Decrease the threshold.
  warnings.warn(


## AffinityPropagation

<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_affinity_propagation_001.png" width="300">

AffinityPropagation creates clusters by sending messages between pairs of samples until convergence. A dataset is then described using a small number of exemplars, which are identified as those most representative of other samples. The messages sent between pairs represent the suitability for one sample to be the exemplar of the other, which is updated in response to the values from other pairs. This updating happens iteratively until convergence, at which point the final exemplars are chosen, and hence the final clustering is given.



In [8]:
df = pd.DataFrame(data=raw_coords, columns=['lan', 'lon'])

In [9]:
df

,lan,lon
0,53.708705,91.447528
1,53.708743,91.447706
2,53.709851,91.446136
3,53.710516,91.446795
4,53.710323,91.447648
...,...,...
3926605,53.724629,91.456166
3926606,53.724917,91.456224
3926607,53.724752,91.455956
3926608,53.724670,91.455986


In [10]:
from sklearn.cluster import AffinityPropagation

In [35]:
pairs = df.apply(list, axis=1)

In [37]:
list(pairs.sample(10))

[[53.658468322350004, 91.47818389605],
 [53.72484682595, 91.37874488565001],
 [53.8276699158, 91.32422693955],
 [53.69521318345, 91.40062497139999],
 [53.72793215585, 91.44660833835],
 [53.7369598663, 91.4355140099],
 [53.7171570415, 91.40603309885],
 [53.8082498933, 91.32635455440001],
 [53.73118329895, 91.44123942235001],
 [53.7330141085, 91.4447002718]]

In [49]:
afPr = AffinityPropagation().fit(list(pairs.sample(10000)))

c:\Temp\TTE\env\lib\site-packages\sklearn\cluster\_affinity_propagation.py:250: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  warnings.warn(


In [50]:
afPr.cluster_centers_

array([], shape=(0, 2), dtype=float64)